In [26]:
import numpy as np
import pandas as pd

In [27]:
class binary():
    def __init__(self,capas=1,nodos=1):
        self.capas = capas
        self.nodos = nodos
        self.forma = (self.capas,self.nodos)
    def inicializar(self):
        self.red = []
        for capa in range(self.capas):
            w = np.random.normal(size = (self.nodos,self.nodos))
            b = np.random.normal(size = self.nodos)
            self.red.append([w,b])
        w_f = np.random.normal(size = self.nodos)
        b_f = np.random.normal(size = 1)
        self.red.append([w_f,b_f])
    def mutar(self,alfa=0.1):
        red = []
        for r in self.red:
            r0 = r[0]+alfa*np.random.normal(size = r[0].shape)
            r1 = r[1]+alfa*np.random.normal(size = r[1].shape)
            red.append([r0,r1])
        hijo = binary(self.capas,self.nodos)
        hijo.inicializar()
        hijo.red = red
        return hijo
    def cruzar(self,other):
        if self.forma == other.forma:
            hijo = binary(self.capas,self.nodos)
            hijo.inicializar()
            red = []
            for i in range(len(self.red)):
                r0 = (self.red[i][0] + other.red[i][0])/2.0
                r1 = (self.red[i][1] + other.red[i][1])/2.0
                red.append([r0,r1])
            hijo.red = red
            return hijo
        else:
            return None
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    def predecir(self,entrada):
        for r in self.red:
            entrada = np.dot(entrada,r[0])
            entrada = entrada + r[1]
            entrada = self.sigmoid(entrada)
        entrada = entrada.round()
        entrada = int(entrada)
        return entrada
    def evaluar(self,entrada,resultado):
        entrada = self.predecir(entrada)
        if entrada == resultado:
            return 1
        return 0     
            
    

In [28]:
def crear_poblacion(poblacion_por_especie,especies,nodos):
    poblacion = []
    for i in range(poblacion_por_especie):
        for j in especies:
            individuo = binary(j,nodos)
            individuo.inicializar()
            poblacion.append(individuo)
    return poblacion

In [29]:
def evaluar_poblacion(poblacion,muestra):
    scores = []
    for p in poblacion:
        score = 0
        count = 0
        for r in muestra.iterrows():
            v = r[1].target
            u = r[1]
            del u['target']
            score += p.evaluar(u.values,v)
            count += 1
        score = float(score / float(count))
        scores.append(score)
    return scores

In [30]:
def predecir(best_model,test):
    scores = []
    for r in test.iterrows():
        u = r[1]
        score = p.predecir(u)
        scores.append(score)
    return scores

In [31]:
titanic = pd.read_csv('train.csv')
titanic.groupby('Embarked').size()

Embarked
C    168
Q     77
S    644
dtype: int64

In [32]:
titanic = pd.read_csv('train.csv')
titanic = titanic[['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
titanic.loc[titanic.Embarked=='C','Embarked']=-1
titanic.loc[titanic.Embarked=='Q','Embarked']=0
titanic.loc[titanic.Embarked=='S','Embarked']=1
titanic['target'] = titanic.Survived
titanic['Pclass_1']=np.where(titanic.Pclass==1,1,0)
titanic['Pclass_2']=np.where(titanic.Pclass==2,1,0)
titanic['Pclass_3']=np.where(titanic.Pclass==3,1,0)
titanic['Sex']=np.where(titanic.Sex=='male',1,0)
titanic = titanic.fillna(titanic.mean())
titanic[['Age']] = titanic[['Age']]/titanic[['Age']].mean()
titanic[['Fare']] = titanic[['Fare']]/titanic[['Fare']].mean()
del titanic['Pclass'],titanic['Survived']
len(titanic)

891

In [33]:
titanic_train = titanic[titanic.target==0].sample(250)\
                .append(titanic[titanic.target==1].sample(250))
titanic_test = titanic.loc[~titanic.index.isin(titanic_train.index),:]



In [34]:
poblacion = crear_poblacion(1000,[0],9)
poblacion = pd.DataFrame(list(zip(poblacion,range(len(poblacion)))))
poblacion.columns = ['model','score']

In [35]:
alfa = 1
for i in range(1000):
    alfa = np.random.random()
    scores = evaluar_poblacion(poblacion.model.tolist(),titanic_train.sample(200))
    poblacion.score = scores
    print(i,poblacion.score.mean(),poblacion.score.max(),alfa)
    poblacion = poblacion.sort_values('score',ascending=False)[0:int(len(poblacion)/2)]
    nueva_poblacion = []
    alfa = 1
    for p in poblacion.model.tolist():
        hijo = p.mutar(alfa = alfa)
        nueva_poblacion.append(hijo)
    nueva_poblacion = pd.DataFrame(list(zip(nueva_poblacion,range(len(nueva_poblacion)))))
    nueva_poblacion.columns = ['model','score']
    poblacion = poblacion.append(nueva_poblacion).reset_index(drop=True)


0 0.5055149999999999 0.775 2
1 0.5501699999999999 0.79 1
2 0.5723250000000001 0.755 1
3 0.619695 0.82 1
4 0.618385 0.785 1
5 0.64257 0.775 1
6 0.641145 0.76 1
7 0.6343899999999999 0.76 1
8 0.666355 0.795 1
9 0.69497 0.815 1
10 0.6970749999999999 0.81 1
11 0.7088 0.815 1
12 0.718395 0.815 1
13 0.7508250000000001 0.82 1
14 0.699655 0.78 1
15 0.745895 0.825 1
16 0.7591749999999999 0.84 1
17 0.7166800000000001 0.795 1
18 0.7718200000000001 0.84 1
19 0.7664500000000001 0.825 1
20 0.741815 0.8 1
21 0.7420800000000001 0.79 1
22 0.7516499999999999 0.795 1
23 0.77475 0.835 1
24 0.75751 0.805 1
25 0.7460999999999999 0.79 1
26 0.7860900000000001 0.835 1
27 0.7596200000000001 0.815 1
28 0.79271 0.835 1
29 0.7911500000000001 0.83 1
30 0.80489 0.85 1
31 0.80285 0.85 1
32 0.77139 0.805 1
33 0.77515 0.81 1
34 0.76406 0.8 1
35 0.75003 0.785 1
36 0.80713 0.84 1
37 0.76479 0.795 1
38 0.78016 0.825 1
39 0.79167 0.83 1
40 0.81499 0.85 1
41 0.7963250000000001 0.83 1
42 0.783865 0.83 1
43 0.75909 0.79 1
44 0

342 0.76213 0.78 1
343 0.7592450000000001 0.77 1
344 0.768125 0.78 1
345 0.8256850000000001 0.845 1
346 0.78147 0.8 1
347 0.830845 0.84 1
348 0.784415 0.795 1
349 0.768765 0.785 1
350 0.7992600000000002 0.815 1
351 0.8040050000000001 0.815 1
352 0.8211799999999999 0.84 1
353 0.824665 0.83 1
354 0.8099399999999999 0.825 1
355 0.804635 0.815 1
356 0.807665 0.82 1
357 0.7832699999999999 0.795 1
358 0.828465 0.84 1
359 0.784625 0.795 1
360 0.850585 0.865 1
361 0.8383550000000002 0.85 1
362 0.8083599999999999 0.815 1
363 0.79828 0.81 1
364 0.8195749999999998 0.83 1
365 0.759475 0.77 1
366 0.8491400000000001 0.855 1
367 0.7311749999999999 0.755 1
368 0.794945 0.805 1
369 0.78855 0.8 1
370 0.8038200000000002 0.815 1
371 0.7634850000000001 0.78 1
372 0.7992650000000001 0.81 1
373 0.7835650000000001 0.795 1
374 0.8037350000000001 0.815 1
375 0.8053449999999999 0.82 1
376 0.7555449999999999 0.77 1
377 0.79877 0.81 1
378 0.7750699999999999 0.785 1
379 0.7854549999999999 0.795 1
380 0.791755 0.8 1

KeyboardInterrupt: 

In [36]:
scores = evaluar_poblacion(poblacion.model.tolist(),titanic_train)
poblacion.score = scores
poblacion.sort_values('score',ascending=False)

,model,score
164,<__main__.binary object at 0x7f7d33776f28>,0.806
493,<__main__.binary object at 0x7f7d33781a90>,0.806
457,<__main__.binary object at 0x7f7d337760b8>,0.806
50,<__main__.binary object at 0x7f7d336738d0>,0.806
6,<__main__.binary object at 0x7f7d33774e80>,0.806
453,<__main__.binary object at 0x7f7d337b60f0>,0.806
495,<__main__.binary object at 0x7f7d339e00f0>,0.806
412,<__main__.binary object at 0x7f7d33656438>,0.804
407,<__main__.binary object at 0x7f7d33656780>,0.804
246,<__main__.binary object at 0x7f7d3365df98>,0.804


In [37]:
best_model = poblacion.sort_values('score',ascending=False).iloc[0,0]

In [38]:
titanic = pd.read_csv('test.csv')
titanic = titanic[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
titanic.loc[titanic.Embarked=='C','Embarked']=-1
titanic.loc[titanic.Embarked=='Q','Embarked']=0
titanic.loc[titanic.Embarked=='S','Embarked']=1
titanic['Pclass_1']=np.where(titanic.Pclass==1,1,0)
titanic['Pclass_2']=np.where(titanic.Pclass==2,1,0)
titanic['Pclass_3']=np.where(titanic.Pclass==3,1,0)
titanic['Sex']=np.where(titanic.Sex=='male',1,0)
titanic = titanic.fillna(titanic.mean())
titanic[['Age']] = titanic[['Age']]/titanic[['Age']].mean()
titanic[['Fare']] = titanic[['Fare']]/titanic[['Fare']].mean()
del titanic['Pclass']
len(titanic)

418

In [39]:
resultado = pd.read_csv('test.csv')
resultado['Survived'] = predecir(best_model,titanic)
resultado.Survived = resultado.Survived.round().astype(int)
resultado[['PassengerId','Survived']].to_csv('result.csv',index=False)